## Loading Libraries

In [24]:
import pandas as pd
import numpy as np
import torch
import evaluate

In [25]:
device = torch.device('cuda')

## Loading Dataset

In [26]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [27]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [28]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [29]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [30]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
validation_dataset = validation_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

### Tokenizing Data

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [32]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT, return_tensors="pt")
  tokenized_batch["labels"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [33]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 7013/7013 [00:01<00:00, 6541.76 examples/s]


#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [34]:
from custom_model import AutoCompositeModel

model = AutoCompositeModel(device).to(device)

In [35]:
# model.my_new_layers[0].weight.dtype

In [36]:
input = torch.tensor(tokenized_test_dataset['input_ids'][:10]).float().to(device)
input.dtype

torch.float32

## Training Setup

### Data Loaders

In [37]:
from torch.utils.data import DataLoader
training_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(tokenized_validation_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_test_dataset, batch_size=16, shuffle=True)

### Loss Function

In [38]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [39]:
# Optimizers specified in the torch.optim package
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [40]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [41]:
from training_own import Trainer, TrainerConfiguration, get_model_params

config = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)

In [42]:
get_model_params(model)

273445

In [43]:
trainer = Trainer(model=model, trainer_configuration=config, input_column='input_ids', output_column='labels')

In [44]:
len(training_loader)

6137

In [45]:
trainer.train_one_epoch(logging_frequency=100, evaluate_when_logging=False)

c:\Users\Pawel\Documents\_projects\Github-Questions-Classification\notebooks\training_own.py:57: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():


 batch 100 training_loss: 1.6400083601474762 training_accuracy: 0.13249999284744263
 batch 200 training_loss: 1.6370729553699492 training_accuracy: 0.12937499582767487
 batch 300 training_loss: 1.6329244923591615 training_accuracy: 0.12937499582767487
 batch 400 training_loss: 1.6193859839439393 training_accuracy: 0.12437500059604645
 batch 500 training_loss: 1.6175179052352906 training_accuracy: 0.1368750035762787
 batch 600 training_loss: 1.6114121413230895 training_accuracy: 0.1274999976158142
 batch 700 training_loss: 1.6063063311576844 training_accuracy: 0.14499999582767487
 batch 800 training_loss: 1.5986646687984467 training_accuracy: 0.14874999225139618
 batch 900 training_loss: 1.598515533208847 training_accuracy: 0.15187498927116394
 batch 1000 training_loss: 1.5900797498226167 training_accuracy: 0.17125000059604645
 batch 1100 training_loss: 1.5829271519184112 training_accuracy: 0.203125
 batch 1200 training_loss: 1.5778765380382538 training_accuracy: 0.24187499284744263
 ba

(1.3501731806993484, tensor(0.4177, device='cuda:0'))